<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#compare performance of different selection methods, crossovers, and mutations

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [1]:
import sys
sys.path.append('..')

In [2]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import combinations
from typing import Callable
from library.algorithms.genetic_algorithms.algorithm import get_best_ind

In [3]:
# Import the relationship matrix as a pandas dataframe for visualization

relationship_matrix_df = pd.read_csv('data/seating_data.csv')
relationship_matrix_df.drop(columns='idx', inplace=True)
relationship_matrix_df

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


In [4]:
# Import the relationship matrix as a numpy array

relationship_matrix = np.loadtxt('data/seating_data.csv', delimiter=',', skiprows=1)
relationship_matrix = relationship_matrix[ : , 1:]
relationship_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]], shape=(64, 64))

---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [5]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                #while any(idx in table for table in tables):
                    #idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    def fitness(self, verbose=False):
        total_happiness = 0
        i = 1
        for table in self.repr:
            # Sum relationship scores for all unique guest pairs at the table
            table_happiness = sum(self.relationship_matrix[a, b] for a, b in combinations(table, 2))
            total_happiness += table_happiness
            if verbose:
                print(f"Table {i} Happiness = {table_happiness}")
                i += 1
        return total_happiness

In [6]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness(verbose=True))

Random solution: [[36, 24, 40, 27, 25, 63, 19, 51], [18, 0, 13, 29, 41, 48, 45, 38], [55, 44, 39, 54, 42, 20, 33, 49], [43, 15, 62, 34, 28, 37, 4, 35], [12, 32, 14, 6, 59, 11, 8, 16], [1, 17, 5, 31, 22, 50, 2, 3], [46, 47, 53, 26, 23, 21, 56, 7], [9, 60, 52, 58, 10, 30, 57, 61]]
Table 1 Happiness = 700.0
Table 2 Happiness = 800.0
Table 3 Happiness = 2800.0
Table 4 Happiness = 4800.0
Table 5 Happiness = 1400.0
Table 6 Happiness = 5300.0
Table 7 Happiness = 2000.0
Table 8 Happiness = 2200.0
Total Fitness: 20000.0


---

### <span style="color:#96d9f0;">3. Selection Algorithms</span>

#### 3.1. Fitness Proportionate Selection

In a typical **fitness proportionate selection** (also known as **roulette wheel selection**), individuals are selected based on their fitness values, with higher fitness values having a greater chance of being chosen. In our optimization problem, however, the fitness values can range from **negative** (indicating conflicts) to **positive** (indicating good relationships between guests).

To adapt the selection process to handle **negative fitness values**, we introduce the following modifications:

1. **Handling Negative Fitness**:
   - Negative fitness values (e.g., -1000) represent undesirable relationships between guests and are valid in our case.
   - However, to ensure that these negative values don't completely block a solution from being selected, we **normalize** the fitness values by adding an **offset** to make all fitness values non-negative.
   - The offset is the **absolute value of the minimum fitness** if it's negative, ensuring that all fitness values become positive or zero without altering the relative fitness ranking of solutions.

2. **Fitness Normalization**:
   - For each individual, we add the **offset** (if necessary) to their fitness value to ensure it's non-negative.
   - This ensures that individuals with negative fitness still have a chance to be selected but are less likely to be chosen compared to those with higher (positive) fitness.

3. **Roulette Wheel Selection**:
   - After normalizing the fitness values, the selection process proceeds as usual, with higher fitness solutions having a higher probability of being selected.
   - By adding the offset, we make sure that **negative fitness values** don't result in zero or near-zero chances of selection, allowing them to contribute to the evolutionary process.

In [7]:
def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    if maximization:
        fitness_values = []
        min_fitness = min(ind.fitness() for ind in population)  # Find the minimum fitness value
        
        # Add a constant offset to ensure no negative fitness values
        offset = abs(min_fitness) if min_fitness < 0 else 0
        
        # Add the offset to each fitness value
        for ind in population:
            fitness_value = ind.fitness() + offset
            fitness_values.append(fitness_value)
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    
    # For each individual, check if random number is inside the individual's "box"
    box_boundary = 0
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)

In [8]:
population = [WSSolution() for _ in range(10)]

fitness_proportionate_selection_result = fitness_proportionate_selection(population, maximization=True)
print('Fitness Proportionate selection result:', fitness_proportionate_selection_result)

Fitness Proportionate selection result: [[22, 18, 31, 32, 44, 24, 61, 38], [19, 51, 23, 35, 50, 59, 14, 58], [1, 49, 0, 3, 27, 4, 10, 5], [33, 26, 9, 63, 12, 2, 37, 30], [57, 16, 55, 43, 53, 6, 60, 52], [36, 13, 47, 8, 40, 15, 34, 56], [54, 41, 46, 62, 21, 29, 45, 39], [11, 17, 25, 48, 7, 42, 28, 20]]


#### 3.2. Ranking Selection

With Ranking Selection, instead of selecting based on raw fitness values, individuals are ranked by their fitness, and higher-ranked individuals (better seating arrangements) have a higher chance of being selected. This method ensures that negative fitness values aren´t automatically excluded, as selection is based on relative rank rather than absolute fitness. It maintains diversity by giving lower-ranked individuals a chance to contribute, preventing premature convergence.

In [9]:
def ranking_selection(population, maximization=False):
    n_parents = 1  # Always select one parent per call in the GA

    # Sort population based on fitness
    sorted_pop = sorted(
        population,
        key=lambda sol: sol.fitness(),
        reverse=maximization  # reverse=True if maximizing
    )

    # Assign linear ranks
    n = len(sorted_pop)
    ranks = np.arange(1, n + 1)
    total_rank = np.sum(ranks)
    probabilities = ranks / total_rank

    # If maximizing, best is first → reverse probabilities so high rank = high prob
    if maximization:
        probabilities = probabilities[::-1]

    selected = np.random.choice(sorted_pop, size=n_parents, p=probabilities, replace=True)
    return selected[0]  # Return single selected parent


In [10]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")


Fitness of individuals in population:
Individual 0: Fitness = 9300.00
Individual 1: Fitness = 14800.00
Individual 2: Fitness = 9400.00
Individual 3: Fitness = 9300.00
Individual 4: Fitness = 14600.00
Individual 5: Fitness = 10700.00
Individual 6: Fitness = 19500.00
Individual 7: Fitness = 9500.00
Individual 8: Fitness = 14700.00
Individual 9: Fitness = 15800.00

Ranking selection result:
Selected 1: [[20, 47, 51, 21, 19, 34, 12, 38], [58, 60, 49, 48, 59, 18, 28, 63], [45, 53, 11, 39, 41, 33, 26, 17], [40, 1, 62, 50, 30, 22, 56, 14], [8, 42, 36, 16, 29, 43, 7, 35], [55, 4, 54, 15, 13, 24, 61, 5], [44, 23, 46, 10, 31, 27, 2, 32], [0, 52, 57, 25, 6, 37, 3, 9]], Fitness = 14800.00
Selected 2: [[49, 16, 3, 2, 63, 1, 11, 35], [48, 26, 19, 52, 21, 27, 56, 57], [44, 17, 33, 55, 45, 51, 43, 20], [4, 41, 10, 15, 61, 14, 42, 9], [5, 12, 47, 0, 22, 54, 62, 37], [32, 13, 31, 28, 29, 38, 18, 36], [30, 6, 34, 39, 7, 23, 53, 24], [8, 59, 25, 46, 50, 60, 58, 40]], Fitness = 19500.00


In this case, we can see that the same individual (2nd best in rank) was selected twice. This is okay and makes that it coud happen especially with the high rank and small population.

In [11]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")

Fitness of individuals in population:
Individual 0: Fitness = 7500.00
Individual 1: Fitness = 14700.00
Individual 2: Fitness = 13600.00
Individual 3: Fitness = 10800.00
Individual 4: Fitness = 8600.00
Individual 5: Fitness = 8800.00
Individual 6: Fitness = 13400.00
Individual 7: Fitness = 11600.00
Individual 8: Fitness = 11800.00
Individual 9: Fitness = 15500.00

Ranking selection result:
Selected 1: [[60, 17, 15, 62, 63, 14, 39, 26], [34, 41, 9, 42, 30, 20, 21, 31], [43, 7, 19, 10, 4, 53, 35, 8], [57, 59, 47, 61, 5, 33, 44, 38], [52, 16, 3, 11, 46, 22, 36, 55], [6, 32, 48, 50, 49, 51, 56, 23], [1, 27, 18, 29, 58, 13, 40, 24], [25, 12, 2, 28, 54, 0, 45, 37]], Fitness = 10800.00
Selected 2: [[37, 53, 57, 45, 51, 29, 9, 32], [12, 8, 26, 46, 5, 33, 44, 54], [15, 31, 4, 28, 22, 18, 61, 35], [36, 30, 7, 10, 20, 62, 3, 47], [14, 17, 60, 27, 6, 25, 59, 16], [19, 43, 40, 23, 34, 38, 42, 11], [2, 48, 13, 49, 41, 50, 55, 56], [58, 1, 52, 63, 39, 24, 21, 0]], Fitness = 13600.00


#### 3.3. Tournament Selection

The tournament selection randomly chooses 𝑘 individuals at random from the population, compares their fitnesses, and returns the best individual out of those to be a parent. 

In [12]:
def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

In [13]:
population = [WSSolution() for _ in range(10)]

tournament_selection_result = tournament_selection(population)
print('Tournament selection result:', tournament_selection_result)

Tournament selection result: [[38, 55, 13, 59, 15, 54, 5, 14], [57, 4, 62, 44, 31, 51, 45, 49], [40, 36, 0, 2, 11, 32, 29, 6], [16, 19, 43, 63, 10, 41, 17, 12], [18, 47, 60, 24, 23, 42, 22, 8], [48, 50, 33, 30, 25, 27, 56, 46], [26, 61, 37, 3, 9, 20, 21, 53], [28, 58, 7, 1, 52, 34, 39, 35]]


To make sure that our tournament_selection implementation truly picks the fittest individual, we wrapped it in a helper function, tournament_verification, which prints out every step of the process. This function is needed because the core algorithm samples competitors internally, not allowing us to check the candidates considered or their fitness except for the winner that is returned. In tournament_verification, we set 
k=10 and explicitly list all ten randomly chosen competitors along with their fitness values, then identify and print the one selected as the fittest by tournament_selection, allowing us to directly compare that result against the ten and confirm it truly has the highest fitness. This gives us clear, reproducible evidence that the tournament selection returns the strongest contender.

In [14]:
def tournament_verification(population, k=10):
    competitors = random.sample(population, k)
    
    print("Fitness of competitors:")
    for i in range(k):
        print(f"Competitor {i+1}: Fitness = {competitors[i].fitness()}")
    
    best_idx, best_competitor = max(enumerate(competitors),key=lambda pair: pair[1].fitness())
    
    print("\nTournament selection result:")
    print(f"Competitor {best_idx+1}: Fitness = {best_competitor.fitness():.2f}")

In [15]:
population = [WSSolution() for _ in range(50)]

tournament_verification(population)

Fitness of competitors:
Competitor 1: Fitness = 10200.0
Competitor 2: Fitness = 7500.0
Competitor 3: Fitness = 8300.0
Competitor 4: Fitness = 9400.0
Competitor 5: Fitness = 9000.0
Competitor 6: Fitness = 12800.0
Competitor 7: Fitness = 10600.0
Competitor 8: Fitness = 10400.0
Competitor 9: Fitness = 11600.0
Competitor 10: Fitness = 9400.0

Tournament selection result:
Competitor 6: Fitness = 12800.00


---

### <span style="color:#96d9f0;">4. Crossover Operators</span>

#### 4.1. Custom Crossover Operator (`table_level_crossover`)

In this section, we implement a custom crossover operator tailored to the constraints of our optimization problem. Unlike standard one-point crossover, which is suitable for flat representations, our solution requires a structured approach:

- Each solution is a nested list representing 8 tables with 8 unique guests each.
- Every guest (from 0 to 63) must appear exactly once across all tables.
- A direct crossover risks creating invalid offspring (with duplicated or missing guests).

To address this, our operator performs table-level crossover:

1. A one-point crossover is applied between the two parent solutions at the table level. This can result in invalid intermediate offspring that contain duplicate guests and omit others, violating the uniqueness constraint.

2. The resulting child is then repaired by:

    - Detecting duplicate and missing guests
    - Replacing duplicates with missing ones to restore a valid configuration
    - This ensures all offspring are valid solutions and suitable for use in the genetic algorithm.

In [16]:
def table_level_crossover(parent1, parent2):
    # Deep copies of the parent representations
    p1_repr = deepcopy(parent1.repr)
    p2_repr = deepcopy(parent2.repr)

    # Flatten guest lists
    total_guests = len(parent1.relationship_matrix)

    # One-point crossover at table level
    xo_point = random.randint(1, len(p1_repr) - 1)
    child1_repr = p1_repr[:xo_point] + p2_repr[xo_point:]
    child2_repr = p2_repr[:xo_point] + p1_repr[xo_point:]

    def repair(child_repr):
        # Count how many times each guest appears
        guest_counts = {}
        for table in child_repr:
            for guest in table:
                guest_counts[guest] = guest_counts.get(guest, 0) + 1

        # Identify missing and extra guests
        missing = [g for g in range(total_guests) if guest_counts.get(g, 0) == 0]
        extras = [g for g, count in guest_counts.items() if count > 1 for _ in range(count - 1)]

        # Replace extra guests with missing ones
        for table in child_repr:
            for i in range(len(table)):
                guest = table[i]
                if guest_counts[guest] > 1:
                    new_guest = missing.pop()
                    guest_counts[guest] -= 1
                    table[i] = new_guest
                    guest_counts[new_guest] = 1  # added to the table

        return child_repr

    # Repair both children
    repaired1 = repair(child1_repr)
    repaired2 = repair(child2_repr)

    return WSSolution(repr=repaired1), WSSolution(repr=repaired2)

In [17]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = table_level_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[60, 59, 35, 14, 48, 56, 51, 0], [39, 16, 57, 10, 63, 33, 34, 11], [43, 13, 61, 53, 3, 8, 44, 20], [21, 41, 12, 25, 54, 42, 17, 29], [18, 62, 22, 55, 7, 15, 1, 24], [37, 2, 4, 30, 6, 5, 32, 23], [40, 27, 19, 47, 26, 49, 36, 52], [46, 58, 28, 9, 50, 31, 38, 45]]
Parent 2 Seating Arrangement:
[[44, 27, 58, 24, 38, 54, 48, 39], [28, 35, 10, 3, 23, 13, 25, 49], [37, 9, 14, 40, 63, 0, 31, 41], [56, 29, 43, 33, 52, 11, 46, 60], [1, 21, 34, 61, 30, 20, 17, 18], [55, 53, 57, 6, 62, 19, 47, 12], [8, 59, 45, 22, 50, 15, 16, 36], [7, 5, 51, 2, 32, 4, 26, 42]]

Offspring 1 Seating Arrangement:
[[58, 54, 35, 49, 48, 44, 38, 28], [39, 27, 25, 10, 24, 23, 13, 3], [37, 9, 14, 40, 63, 0, 31, 41], [56, 29, 43, 33, 52, 11, 46, 60], [1, 21, 34, 61, 30, 20, 17, 18], [55, 53, 57, 6, 62, 19, 47, 12], [8, 59, 45, 22, 50, 15, 16, 36], [7, 5, 51, 2, 32, 4, 26, 42]]
Offspring 2 Seating Arrangement:
[[63, 60, 59, 57, 56, 51, 48, 39], [34, 35, 10, 33, 16, 14, 11, 0], [43, 13, 61, 53,

In [18]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.2. Partially Matched Crossover

The Partially Matched Crossover (pmc) is a genetic algorithm operator designed for permutations, like our guest seating arrangement. It swaps a segment between two parent solutions and builds offspring by preserving the order and position of elements while avoiding duplicates. In our implementation, we adapted PMX to work with the wedding seating format by flattening table arrangements into 1D lists, applying PMX, and then converting them back. We carefully resolved conflicts during crossover by mapping duplicate entries to valid unused guests, ensuring that each guest appears exactly once in the final seating—maintaining valid, non-repetitive arrangements.

In [19]:
def partially_matched_crossover(parent1, parent2):
    """
    PMC crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    import random

    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def pmc(p1, p2):
        child = [None] * size
        # Copy the crossover segment from p1
        child[point1:point2] = p1[point1:point2]

        for i in range(point1, point2):
            if p2[i] not in child:
                val = p2[i]
                idx = i
                while True:
                    mapped = p1[idx]
                    idx = p2.index(mapped)
                    if child[idx] is None:
                        child[idx] = val
                        break

        # Fill remaining positions from p2
        for i in range(size):
            if child[i] is None:
                child[i] = p2[i]

        return child

    child1_flat = pmc(parent1_flat, parent2_flat)
    child2_flat = pmc(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2


In [20]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = partially_matched_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[47, 16, 5, 27, 42, 11, 26, 44], [22, 33, 63, 13, 23, 12, 9, 40], [54, 32, 51, 58, 31, 57, 19, 56], [24, 20, 21, 30, 43, 49, 7, 10], [18, 59, 29, 34, 28, 39, 60, 6], [3, 45, 50, 41, 36, 55, 35, 37], [2, 15, 61, 14, 62, 38, 0, 53], [48, 4, 46, 17, 1, 8, 52, 25]]
Parent 2 Seating Arrangement:
[[22, 38, 39, 24, 2, 33, 42, 8], [23, 62, 31, 0, 36, 48, 41, 17], [9, 57, 32, 12, 18, 19, 16, 53], [44, 7, 5, 28, 37, 15, 60, 43], [6, 52, 20, 51, 29, 1, 50, 21], [4, 63, 11, 55, 13, 61, 56, 27], [40, 59, 10, 47, 45, 14, 30, 58], [25, 49, 3, 34, 46, 54, 26, 35]]

Offspring 1 Seating Arrangement:
[[0, 38, 1, 8, 17, 5, 26, 44], [22, 33, 63, 13, 23, 12, 9, 40], [54, 32, 51, 58, 31, 57, 19, 56], [24, 20, 21, 30, 43, 49, 7, 10], [18, 59, 29, 34, 28, 39, 60, 6], [3, 45, 50, 41, 36, 55, 35, 37], [2, 15, 61, 14, 62, 47, 11, 48], [25, 52, 4, 16, 46, 27, 42, 53]]
Offspring 2 Seating Arrangement:
[[14, 34, 33, 54, 26, 30, 42, 8], [23, 62, 31, 0, 36, 48, 41, 17], [9, 57, 32, 12, 1

In [21]:
# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.3. Order Crossover

The Order Crossover (OX) is a genetic algorithm operator also designed for permutation problems, such as our guest seating arrangements. It selects a contiguous block of seats from one parent and preserves both the relative order and positional integrity of the remaining guests by filling in, in sequence, from the other parent’s list and skipping any guest already included. In our implementation, we adapt OX to the wedding format by flattening each table layout into a single list, choosing two crossover points, copying the segment from Parent 1 into Offspring 1 (and vice versa for Offspring 2), then rotating through the other parent’s flattened list to fill the empty slots while avoiding duplicates. Finally, we reshape the 1D offspring lists back into the original table structure, guaranteeing a complete, non-repetitive seating plan.

In [22]:
def order_crossover(parent1, parent2):
    """
    Order crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def order(p1, p2):
        child = [None] * size
        child[point1:point2] = p1[point1:point2]
        
        for i in range(point2-size, point1):
            val = p2[i]
            idx = i
            while val in child:
                idx+=1
                val = p2[idx]
            child[i] = val

        return child

    child1_flat = order(parent1_flat, parent2_flat)
    child2_flat = order(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2

In [23]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply Order crossover
offspring1, offspring2 = order_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())

Parent 1 Seating Arrangement:
[[10, 2, 15, 36, 22, 25, 49, 32], [29, 39, 9, 46, 6, 60, 40, 8], [62, 30, 17, 19, 54, 35, 38, 18], [51, 55, 37, 28, 23, 7, 47, 1], [59, 0, 63, 21, 57, 53, 45, 12], [44, 48, 43, 34, 26, 33, 56, 14], [24, 61, 50, 58, 27, 16, 5, 11], [52, 13, 31, 20, 42, 4, 3, 41]]
Parent 2 Seating Arrangement:
[[61, 19, 18, 28, 58, 60, 56, 37], [49, 23, 9, 13, 29, 62, 25, 22], [41, 54, 36, 17, 48, 53, 7, 5], [12, 16, 1, 50, 0, 51, 45, 43], [40, 14, 15, 4, 35, 59, 26, 34], [8, 46, 55, 38, 57, 27, 6, 44], [47, 3, 21, 31, 39, 20, 33, 63], [11, 32, 2, 30, 42, 10, 52, 24]]

Offspring 1 Seating Arrangement:
[[56, 49, 9, 13, 29, 62, 25, 22], [41, 54, 36, 17, 5, 16, 50, 40], [14, 15, 4, 35, 8, 46, 38, 18], [51, 55, 37, 28, 23, 7, 47, 1], [59, 0, 63, 21, 57, 53, 45, 12], [44, 48, 43, 34, 26, 27, 6, 3], [31, 39, 20, 33, 11, 32, 2, 30], [42, 10, 52, 24, 61, 19, 58, 60]]
Offspring 2 Seating Arrangement:
[[49, 32, 29, 39, 9, 6, 60, 62], [30, 17, 19, 54, 18, 37, 28, 23], [7, 47, 63, 21, 5

In [24]:
# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


---

### <span style="color:#96d9f0;">5. Mutation Operators</span>

#### 5.1. Swap Mutation

The swap mutation operator introduces random changes to the seating arrangement by swapping two guests between different tables.

It's important to note that swapping two guests within the same table would not create any new configuration - it would simply result in the same solution as before. Since we want to introduce meaningful changes, the swap mutation ensures that the two guests are always swapped between different tables. This prevents the mutation from generating an invalid or identical solution.

This mutation helps the genetic algorithm explore new seating arrangements by shuffling guests across tables while respecting the overall constraints of the problem.

In [25]:
def swap_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    if len(tables) < 2:
        return new_solution

    # Pick two random tables
    t1, t2 = random.sample(range(len(tables)), 2)
    if not tables[t1] or not tables[t2]:
        return new_solution

    # Swap one person from each
    i1 = random.randint(0, len(tables[t1]) - 1)
    i2 = random.randint(0, len(tables[t2]) - 1)
    tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

    return new_solution

In [26]:
solution = WSSolution()

mutated_solution = swap_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[30, 60, 28, 43, 20, 11, 36, 39], [12, 44, 24, 4, 17, 47, 1, 58], [5, 26, 56, 16, 46, 52, 23, 21], [10, 38, 9, 15, 48, 53, 32, 6], [37, 55, 27, 62, 3, 31, 57, 14], [0, 51, 8, 19, 45, 59, 13, 34], [18, 2, 41, 49, 22, 25, 54, 63], [33, 40, 50, 61, 42, 35, 7, 29]]
Mutated solution: [[30, 60, 28, 43, 20, 11, 36, 39], [12, 44, 24, 4, 17, 47, 1, 58], [5, 26, 56, 16, 46, 52, 23, 21], [10, 38, 9, 15, 48, 53, 32, 6], [37, 55, 27, 62, 3, 31, 57, 14], [0, 51, 8, 19, 45, 59, 42, 34], [18, 2, 41, 49, 22, 25, 54, 63], [33, 40, 50, 61, 13, 35, 7, 29]]


#### 5.2. Inversion Mutation

The inversion mutation operator introduces variation by selecting a random segment of the seating plan and reversing the order of guests within that segment.

Since the seating arrangement is represented as a list of tables, the entire structure is first flattened into a single list containing all guests. The inversion is then performed on a random sublist, effectively inverting a continuous sequence of guests. After the inversion, the list is reshaped back into the original table format.

This mutation allows the algorithm to explore new configurations by reordering guests while preserving the structure of the solution. To ensure validity, a repair function is applied after mutation to guarantee that all guests remain uniquely assigned with no duplicates or omissions.

In [27]:
def inversion_mutation_seating(individual: WSSolution, mut_prob):
    # Access the seating arrangement (8 tables with guests)
    tables = individual.repr

    rows = len(tables)
    cols = len(tables[0])
    
    # Flatten the seating arrangement into a single list of guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation with the given probability
    if random.random() < mut_prob:
        # Select two random indices for the inversion
        start_row, end_row = sorted(random.sample(range(rows), 2))
        start_col, end_col = random.sample(range(cols), 2)
        start = start_row * cols + start_col
        end = end_row * cols + end_col
        # Reverse the segment between the two indices
        flat[start:end + 1] = reversed(flat[start:end + 1])

    # Reshape the flat list back into 8 tables of 8 guests each
    new_tables = [flat[i:i + individual.table_capacity] for i in range(0, len(flat), individual.table_capacity)]

    # Return a new WSSolution with the mutated seating arrangement
    return WSSolution(repr=new_tables, relationship_matrix=individual.relationship_matrix, nr_of_tables=individual.nr_of_tables)

In [28]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[28, 39, 18, 34, 17, 12, 16, 43], [55, 31, 3, 57, 44, 8, 63, 6], [47, 52, 13, 42, 33, 32, 48, 56], [60, 4, 24, 29, 14, 62, 7, 40], [1, 58, 9, 11, 45, 23, 51, 37], [20, 15, 2, 46, 53, 27, 21, 10], [41, 49, 59, 50, 35, 54, 25, 61], [36, 30, 5, 0, 19, 38, 22, 26]]
Mutated solution: [[28, 39, 18, 34, 17, 12, 16, 43], [55, 31, 3, 48, 32, 33, 42, 13], [52, 47, 6, 63, 8, 44, 57, 56], [60, 4, 24, 29, 14, 62, 7, 40], [1, 58, 9, 11, 45, 23, 51, 37], [20, 15, 2, 46, 53, 27, 21, 10], [41, 49, 59, 50, 35, 54, 25, 61], [36, 30, 5, 0, 19, 38, 22, 26]]


#### 5.3. Scramble Mutation

The scramble mutation operator introduces variation by selecting a random subset of guests in the seating plan and scrambling their order within the selected segment. Unlike the inversion mutation, where the order is reversed, the scramble mutation randomizes the order of guests in a selected sublist.

The process begins by flattening the seating arrangement, which is represented as a list of tables, into a single list of guests. Then, a random segment of guests is selected, and the order of the guests within that segment is scrambled. After the scrambling is performed, the list is reshaped back into the original table structure.

This mutation operator allows the algorithm to explore different configurations by rearranging guests within a specified segment of the seating arrangement, offering a broader search of potential solutions. It maintains the structure of the solution by preserving the overall number of guests per table while allowing the order of guests to vary.

To ensure validity and correctness of the solution, a repair function is applied after the mutation to guarantee that all guests remain uniquely assigned, with no duplicates or omissions.

In [29]:
def scramble_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    rows = len(tables)
    cols = len(tables[0])

    if rows < 2:
        return new_solution

    selected_cols = [random.randint(0, cols-1) for _ in range(rows)]
    selected_positions = [(row, selected_cols[row]) for row in range(rows)]

    selected_values = []
    for row, col in selected_positions: 
        if not tables[row][col]:
            return new_solution
        else: 
            selected_values.append(tables[row][col])
    
    def deranged_shuffle(lst):
        while True:
            shuffled = lst[:] 
            random.shuffle(shuffled)
            if all(a != b for a, b in zip(lst, shuffled)):
                return shuffled
    
    selected_values = deranged_shuffle(selected_values)

    for (row, col), value in zip(selected_positions, selected_values):
        tables[row][col] = value

    return new_solution

In [30]:
solution = WSSolution()

mutated_solution = scramble_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[54, 53, 49, 11, 28, 59, 5, 4], [2, 26, 19, 22, 39, 40, 63, 51], [20, 27, 8, 32, 24, 48, 42, 6], [7, 33, 61, 31, 15, 34, 9, 14], [23, 1, 50, 62, 37, 13, 16, 38], [45, 58, 17, 18, 30, 10, 25, 43], [56, 12, 47, 0, 35, 46, 3, 36], [52, 41, 29, 55, 57, 21, 44, 60]]
Mutated solution: [[54, 53, 49, 11, 28, 59, 5, 4], [2, 26, 19, 22, 39, 40, 63, 51], [20, 27, 8, 32, 24, 48, 42, 6], [7, 33, 61, 31, 15, 34, 9, 14], [23, 1, 50, 62, 37, 13, 16, 38], [45, 58, 17, 18, 30, 10, 25, 43], [56, 12, 47, 0, 35, 46, 3, 36], [52, 41, 29, 55, 57, 21, 44, 60]]


### <span style="color:#96d9f0;">6. Random Trials</span>

In [31]:
import time

start = time.time()
for i in range(10):
    res = 0
    scores = []
    for j in range(100000):
        solution = WSSolution()
        res=solution.fitness()
        scores.append(res)
    avg_score = sum(scores)/100000
    best_score = max(scores)
    print(f"Result {i+1}: Average Fitness: {avg_score} and Best Fitness Found {best_score}")

end = time.time()

print(f"Time elapsed: {int((end- start)//60)} minutes and {round((end-start)%60)} seconds")

KeyboardInterrupt: 

If 1000000 solutions took around 2 minutes, then the total seating arrangements (assuming a much simpler scenario where all random solutions displayed never repeat) would be:

In [4]:
import math

total_combinations = (math.comb(64,8)*math.comb(56,8)*math.comb(48, 8)*math.comb(40, 8)*math.comb(32, 8)*math.comb(24, 8)*math.comb(16,8)) / math.factorial(8)
minutes_taken = total_combinations * 2 / 1000000
years_taken = minutes_taken/(60*24*365) 


print(f" There are {total_combinations} different seating arrangements and it would take around {years_taken} years for a computer to compare all the solutions and find the best.")

 There are 4.505387879868752e+47 different seating arrangements and it would take around 1.7143789497217472e+36 years for a computer to compare all the solutions and find the best.


---

### <span style="color:#96d9f0;">7. Genetic Algorithm</span>

In [31]:
def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    crossover_operator: Callable,
    mutation_operator: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes a genetic algorithm to optimize a population of solutions.

    Args:
        initial_population (list[Solution]): The starting population of solutions.
        max_gen (int): The maximum number of generations to evolve.
        selection_algorithm (Callable): Function used for selecting individuals.
        crossover_operator (Callable): Function to perform crossover between two individuals.
        mutation_operator (Callable): Function to mutate an individual.
        maximization (bool, optional): If True, maximizes the fitness function; otherwise, minimizes. Defaults to False.
        xo_prob (float, optional): Probability of applying crossover. Defaults to 0.9.
        mut_prob (float, optional): Probability of applying mutation. Defaults to 0.2.
        elitism (bool, optional): If True, carries the best individual to the next generation. Defaults to True.
        verbose (bool, optional): If True, prints detailed logs for debugging. Defaults to False.

    Returns:
        Solution: The best solution found after evolving for max_gen generations.
    """
    population = initial_population

    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        new_population = []

        if elitism:
            new_population.append(deepcopy(get_best_ind(population, maximization)))

        while len(new_population) < len(population):
            parent1 = selection_algorithm(population, maximization)
            parent2 = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected parents:\n{parent1}\n{parent2}')

            if random.random() < xo_prob:
                offspring1, offspring2 = crossover_operator(parent1, parent2)
                if verbose:
                    print('Crossover applied.')
            else:
                offspring1, offspring2 = deepcopy(parent1), deepcopy(parent2)
                if verbose:
                    print('Replication applied.')

            mutated1 = mutation_operator(offspring1, mut_prob)
            new_population.append(mutated1)
            if verbose:
                print(f'Mutated offspring 1: {mutated1}')

            if len(new_population) < len(population):
                mutated2 = mutation_operator(offspring2, mut_prob)
                new_population.append(mutated2)
                if verbose:
                    print(f'Mutated offspring 2: {mutated2}')

        population = new_population

        if verbose:
            best = get_best_ind(population, maximization)
            print(f'Best individual fitness this generation: {best.fitness()}')

    return get_best_ind(population, maximization)

In [32]:
# Number of individuals in the initial population
population_size = 50

# Generate initial population of random WSSolutions
population = [WSSolution() for _ in range(population_size)]

In [34]:
best = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best)
print("Fitness:", best.fitness())

In [35]:
isa_try = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(isa_try)
print("Fitness:", isa_try.fitness())

In [36]:
#Number of individuals in the initial population
population_size2 = 3

# Generate initial population of random WSSolutions
population2 = [WSSolution() for _ in range(population_size2)]

best2 = genetic_algorithm(
    initial_population=population2,
    max_gen=5,
    selection_algorithm=tournament_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=inversion_mutation_seating,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best2)
print("Fitness:", best2.fitness())

In [37]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
pmc_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(pmc_test.repr)
print("Fitness:", pmc_test.fitness())

In [62]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(pmc_test.repr))

Offspring 2 has no repeated guests: True


In [38]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
ranking_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(ranking_test.repr)
print("Fitness:", ranking_test.fitness())

In [61]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(ranking_test.repr))

Offspring 2 has no repeated guests: True


---

### <span style="color:#96d9f0;">8. Hill Climbing / Simulated Annealing</span>

In [32]:
class WSSearchableSolution(WSSolution):
    def get_neighbors(self):
        """Generates all valid neighbors by swapping two guests between tables."""
        neighbors = []

        # Loop through all pairs of guests at different tables
        for i in range(len(self.repr)):
            for j in range(len(self.repr[i])):
                for k in range(len(self.repr)):
                    for l in range(len(self.repr[k])):
                        if (i, j) != (k, l):  # Skip swapping guests at the same position
                            new_seating = deepcopy(self.repr)  # Use `repr` which holds the seating
                            new_seating[i][j], new_seating[k][l] = new_seating[k][l], new_seating[i][j]

                            # Create a new WSSearchableSolution with the modified seating arrangement
                            neighbor = WSSearchableSolution(
                                repr=new_seating,
                                relationship_matrix=self.relationship_matrix,  # Use relationship_matrix
                                nr_of_tables=self.nr_of_tables
                            )
                            neighbors.append(neighbor)

        return neighbors

    def get_random_neighbor(self):
        """Generates a random valid neighbor by swapping two guests between different tables."""
        new_seating = deepcopy(self.repr)  # Use `repr` for seating arrangement

        # Randomly pick two different tables
        table_indices = list(range(len(new_seating)))
        t1, t2 = random.sample(table_indices, 2)

        # Make sure the tables are not empty
        if not new_seating[t1] or not new_seating[t2]:
            return deepcopy(self)

        # Randomly pick a guest from each table
        g1 = random.randrange(len(new_seating[t1]))
        g2 = random.randrange(len(new_seating[t2]))

        # Swap the guests
        new_seating[t1][g1], new_seating[t2][g2] = new_seating[t2][g2], new_seating[t1][g1]

        return WSSearchableSolution(
            repr=new_seating,
            relationship_matrix=self.relationship_matrix,  # Use relationship_matrix
            nr_of_tables=self.nr_of_tables
        )

In [33]:
# Meter isto nos imports
from library.algorithms.simulated_annealing import simulated_annealing

def hill_climbing(initial_solution: Solution, maximization=False, max_iter=99999, verbose=False):
    """
    Implementation of the Hill Climbing optimization algorithm.  

    The algorithm iteratively explores the neighbors of the current solution, moving to a neighbor if it improves the objective function.  
    The process continues until no improvement is found or the maximum number of iterations is reached.  

    Args:
        initial_solution (Solution): The starting solution, which must implement the `fitness()` and `get_neighbors()` methods.
        maximization (bool, optional): If True, the algorithm maximizes the fitness function; otherwise, it minimizes it. Defaults to False.
        max_iter (int, optional): The maximum number of iterations allowed before stopping. Defaults to 99,999.
        verbose (bool, optional): If True, prints progress details during execution. Defaults to False.

    Returns:
        Solution: The best solution found during the search.

    Notes:
        - The initial_solution must implement a `fitness()` and `get_neighbors()` method.
        - The algorithm does not guarantee a global optimum; it only finds a local optimum.
    """

    # Run some validations to make sure initial solution is well implemented
    run_validations(initial_solution)

    current = initial_solution
    improved = True
    iter = 1

    while improved:
        if verbose:
            print(f'Current solution: {current} with fitness {current.fitness()}')

        improved = False
        neighbors = current.get_neighbors() # Solution must have a get_neighbors() method

        for neighbor in neighbors:

            if verbose:
                print(f'Neighbor: {neighbor} with fitness {neighbor.fitness()}')

            if maximization and (neighbor.fitness() > current.fitness()):
                current = deepcopy(neighbor)
                improved = True
            elif not maximization and (neighbor.fitness() <= current.fitness()):
                current = deepcopy(neighbor)
                improved = True
        
        iter += 1
        if iter == max_iter:
            break
    
    return current

def run_validations(initial_solution):
    if not isinstance(initial_solution, Solution):
        raise TypeError("Initial solution must be an object of a class that inherits from Solution")
    if not hasattr(initial_solution, "get_neighbors"):
        print(f"The method 'get_neighbors' must be implemented in the initial soltuion.")
    neighbors = initial_solution.get_neighbors()
    if not isinstance(neighbors, list):
        raise TypeError("get_neighbors method must return a list")
    if not all([isinstance(neighbor, type(initial_solution)) for neighbor in neighbors]):
        raise TypeError(f"Neighbors must be of the same type as solution object: {type(initial_solution)}")

In [34]:
# Create a random seating arrangement using WSSolution
initial_solution = WSSolution(relationship_matrix=relationship_matrix, nr_of_tables=8)
seating = initial_solution.random_initial_representation()

# Print initial seating and fitness
print("Initial Seating Arrangement:", seating)
print("Initial Fitness:", initial_solution.fitness())

# Create a searchable solution
searchable_solution = WSSearchableSolution(
    repr=seating,
    relationship_matrix=relationship_matrix,
    nr_of_tables=8
)

# Generate neighbors and print the number of neighbors
neighbors = searchable_solution.get_neighbors()
print(f"Generated {len(neighbors)} neighbors.")

# Test random neighbor generation
random_neighbor = searchable_solution.get_random_neighbor()
print("Random Neighbor Seating:", random_neighbor.repr)
print("Random Neighbor Fitness:", random_neighbor.fitness())

Initial Seating Arrangement: [[29, 8, 27, 51, 6, 46, 19, 31], [10, 28, 53, 38, 35, 15, 50, 36], [57, 2, 25, 40, 32, 9, 14, 12], [17, 43, 13, 48, 41, 52, 24, 26], [39, 61, 21, 7, 62, 1, 22, 34], [4, 55, 16, 33, 56, 63, 54, 49], [45, 44, 58, 5, 30, 11, 60, 0], [59, 37, 18, 47, 20, 3, 42, 23]]
Initial Fitness: 11900.0
Generated 4032 neighbors.
Random Neighbor Seating: [[29, 8, 27, 40, 6, 46, 19, 31], [10, 28, 53, 38, 35, 15, 50, 36], [57, 2, 25, 51, 32, 9, 14, 12], [17, 43, 13, 48, 41, 52, 24, 26], [39, 61, 21, 7, 62, 1, 22, 34], [4, 55, 16, 33, 56, 63, 54, 49], [45, 44, 58, 5, 30, 11, 60, 0], [59, 37, 18, 47, 20, 3, 42, 23]]
Random Neighbor Fitness: 10000.0


In [35]:
# Generate initial random seating using WSSolution
initial_solution = WSSolution(relationship_matrix=relationship_matrix, nr_of_tables=8)
seating = initial_solution.random_initial_representation()

# Wrap the seating in a WSSearchableSolution
searchable_solution = WSSearchableSolution(
    repr=seating,
    relationship_matrix=relationship_matrix,
    nr_of_tables=8
)

# Display initial configuration
print("Initial Seating:", seating)
print("Initial Fitness:", searchable_solution.fitness())

Initial Seating: [[63, 43, 62, 48, 53, 52, 27, 16], [15, 45, 29, 49, 26, 39, 56, 57], [32, 25, 0, 17, 46, 3, 12, 38], [6, 1, 58, 54, 34, 42, 35, 14], [13, 24, 8, 44, 20, 28, 41, 51], [59, 4, 31, 9, 30, 55, 2, 18], [11, 7, 61, 47, 60, 50, 19, 37], [23, 36, 33, 21, 22, 40, 10, 5]]
Initial Fitness: 12700.0


In [40]:
# Run Hill Climbing
print("Running Hill Climbing...")
best_hc_solution = hill_climbing(
    initial_solution=searchable_solution,
    maximization=True,
    max_iter=1,
    verbose=False
)

print("\nBest Hill Climbing Solution:")
print("Seating:", best_hc_solution.repr)
print("Fitness:", best_hc_solution.fitness())

Running Hill Climbing...

Best Hill Climbing Solution:
Seating: [[37, 43, 59, 44, 38, 49, 27, 36], [40, 42, 41, 25, 26, 39, 56, 57], [14, 13, 0, 53, 11, 1, 12, 10], [6, 3, 7, 2, 34, 8, 35, 9], [52, 30, 45, 48, 50, 28, 29, 51], [20, 19, 31, 32, 33, 55, 54, 18], [46, 62, 61, 47, 60, 58, 21, 63], [23, 16, 24, 4, 22, 15, 17, 5]]
Fitness: 77800.0


In [41]:
# Run Simulated Annealing
print("Running Simulated Annealing...")
best_sa_solution = simulated_annealing(
    initial_solution=searchable_solution,
    C=100.0,
    L=20,
    H=1.1,
    maximization=True,
    max_iter=10,
    verbose=False
)

print("\nBest Simulated Annealing Solution:")
print("Seating:", best_sa_solution.repr)
print("Fitness:", best_sa_solution.fitness())

Running Simulated Annealing...

Best Simulated Annealing Solution:
Seating: [[36, 50, 39, 59, 40, 42, 30, 37], [54, 38, 26, 58, 29, 28, 56, 57], [32, 14, 43, 47, 46, 31, 48, 45], [6, 12, 3, 44, 34, 33, 7, 35], [5, 19, 55, 18, 20, 62, 41, 22], [11, 8, 51, 9, 13, 10, 2, 52], [25, 27, 61, 17, 60, 21, 63, 53], [23, 24, 15, 4, 1, 0, 16, 49]]
Fitness: 49000.0
